<h1>T4 - Traçado de Raios<h3/>

<p></p>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import sin, cos, sqrt
import sys

TOL = sys.float_info.epsilon
print(TOL)

2.220446049250313e-16


In [2]:
def vetor(x, y, z):
    return  np.array([x, y, z], dtype = np.float)

def dot(u, v):
    return (u[0] * v[0] + u[1] * v[1] + u[2] * v[2])

def norma(u):
    return sqrt(dot(u, u))

def unitario(v):
    s = norma(v)
    
    if (s > TOL):
        return v/s
    
    else:
        return None

def ang(v1, v2):
    num = dot(v1, v2)
    den = norm(v1) * norm(v2)
    
    return np.arccos(num/den) * 180 / np.pi if den > TOL else 0

def reflete(v, n):
    r = 2 * dot(v, n) * n - v
    return r

def cross(u, v):
    return vetor(u[1] * v[2] - u[2] * v[1], u[2] * v[0] - u[0] * v[2], u[0] * v[1] - u[1] * v[0]) 

In [3]:
class Camera:
    def __init__(self, fov, w, h, dfocal, eye, at, up):
        self.fov = fov
        self.w = w
        self.h = h
        self.dfocal = dfocal
        
        self.a = 2 * dfocal * np.tan(fov * np.pi / 360)
        self.b = self.a * w / h
        self.eye = eye
        
        self.ze = unitario(at - eye)
        self.xe = unitario(cross(self.ze, up))
        self.ye = cross(self.ze, self.xe)
        self.img = np.full(shape = (h, w, 3), fill_value = 0.43, dtype = np.float)
    
    def ray_to(self, x_im, y_im):
        dx = self.b * (x_im / self.w - 0.5)
        dy = self.a * (y_im / self.h - 0.5)
        dz = self.dfocal
        
        ray = dx * self.xe + dy * self.ye + dz * self.ze
        
        return ray
    
    def get_eye(self):
        return self.eye

    def pixel (self, x_im, y_im, rgb):
        self.img[y_im, x_im, :] = rgb
    
    def get_pixel(self, x, y):
        return self.img[y, x, :]
    
    def set_pixel(self, x, y, rgb):
        self.img[y, x, :] = rgb
        
    def get_w(self):
        return self.w
    
    def get_h(self):
        return self.h
    
    def imshow(self):
        img_max = np.amax(self.img)
        
        if img_max < TOL:
            print("Imagem Zerada")
            
        plt.figure(figsize = (8, 4))
        plt.imshow(self.img)
        plt.show()
    
    def show(self):
        print("CAMERA:")
        print("fov =", self.fov, "d =", self.dfocal)
        print("(w, h) = (", self.w, ",", self.h, ")")
        print("(b, a) = (", self.b, ",", self.a, ")")
        print("xe =", self.xe)
        print("ye =", self.ye)
        print("ze =", self.ze)

In [4]:
eye = vetor(100, 40, 40)
at = vetor(0, 0, 0)
up = vetor(0, 1, 0)
camera = Camera(90, 300, 300, 30, eye, at, up)
camera.show()

CAMERA:
fov = 90 d = 30
(w, h) = ( 300 , 300 )
(b, a) = ( 59.999999999999986 , 59.99999999999999 )
xe = [ 0.37139068  0.         -0.92847669]
ye = [ 0.32325409 -0.93743687  0.12930164]
ze = [-0.87038828 -0.34815531 -0.34815531]


In [5]:
class Poligono:
    def __init__(self, pp, pn, material):
        self.pp = pp
        self.pn = pn
        self.ni = None
        self.material = material
    
    def show(self):
        print("POLIGONO:")
        for i in range(len(self.pp)):
            print(f'Plano {i}: ponto = {self.pp[i]}, normal = {self.pn[i]}')
    
    def intercepta(self, origem, direcao):
        te = 0
        ts = np.inf
        obji = None
        
        for i in range(len(self.pp)):
            pn = self.pn[i]
            pp = self.pp[i]
            num = dot(pp - origem, pn)
            den = dot(direcao, pn)
            
            if den < -TOL:
                t = num / den
                
                if t > te:
                    te = t
                    obji = self
                    self.ni = pn
                
            elif den > TOL:
                t = num / den
                ts = t if t < ts else ts
                obji = self
            
            if ts < te:
                te = np.inf
                obji = None
                break
        
        return te, obji
    
    def get_mat(self):
        return self.material
    
    def normal(self, ponto):
        return self.pn

In [6]:
class Caixa(Poligono):
    def __init__(self, pmin, pmax, material):
        super().__init__([pmin, pmin, pmin, pmax, pmax, pmax], 
                         [vetor(-1, 0 , 0), vetor(0, -1, 0), vetor(0, 0, -1), 
                          vetor(1, 0 , 0), vetor(0, 1, 0), vetor(0, 0, 1)], material)

In [7]:
class Esfera:
    def __init__(self, raio, centro, material):
        self.raio = raio
        self.centro = centro
        self.material = material
        
    def show(self):
        print("ESFERA:")
        print(f'raio = {self.raio}')
        print(f'centro = {self.centro}')
        self.material.show()
    
    def intercepta(self, origem, direcao):
        a = dot(direcao, direcao)
        b = 2 * dot(direcao, origem - self.centro)
        c = dot(origem - self.centro, origem - self.centro) - self.raio**2
        delta = b**2 - 4 * a * c
        
        if delta > TOL:
            raiz = sqrt(delta)
            t1 = (-b - raiz) / (2 * a)
            t2 = (-b + raiz) / (2 * a)
            t = t1 if t1 < t2 else t2
            
            if t > 0:
                return t, self
            else:
                return np.inf, None
        else:
            return np.inf, None
     
    def normal(self, ponto):
        return unitario(ponto - self.centro)
    
    def get_mat(self):
        return self.material

In [8]:
class Material:
    def __init__(self, coef_difuso):
        self.coef_difuso = coef_difuso
        
    def show(self):
        print("MATERIAL:")
        print("kd", self.coef_difuso)
    
    def get_kd(self):
        return self.coef_difuso

In [9]:
azul = Material(vetor(0, 0, 1))
amarelo = Material(vetor(0.7, 0.7, 0))
azul.show()
amarelo.show()

MATERIAL:
kd [0. 0. 1.]
MATERIAL:
kd [0.7 0.7 0. ]


In [10]:
esfera = Esfera(25, vetor(0, 20, 0), azul)
esfera.show()

ESFERA:
raio = 25
centro = [ 0. 20.  0.]
MATERIAL:
kd [0. 0. 1.]


In [11]:
piso = Caixa(vetor(-80, -50, -50), vetor(50, -45, 50), amarelo)
parede = Caixa(vetor(-80, -50, -60), vetor(50, 50, -50), amarelo)
piso.show()
parede.show()

POLIGONO:
Plano 0: ponto = [-80. -50. -50.], normal = [-1.  0.  0.]
Plano 1: ponto = [-80. -50. -50.], normal = [ 0. -1.  0.]
Plano 2: ponto = [-80. -50. -50.], normal = [ 0.  0. -1.]
Plano 3: ponto = [ 50. -45.  50.], normal = [1. 0. 0.]
Plano 4: ponto = [ 50. -45.  50.], normal = [0. 1. 0.]
Plano 5: ponto = [ 50. -45.  50.], normal = [0. 0. 1.]
POLIGONO:
Plano 0: ponto = [-80. -50. -60.], normal = [-1.  0.  0.]
Plano 1: ponto = [-80. -50. -60.], normal = [ 0. -1.  0.]
Plano 2: ponto = [-80. -50. -60.], normal = [ 0.  0. -1.]
Plano 3: ponto = [ 50.  50. -50.], normal = [1. 0. 0.]
Plano 4: ponto = [ 50.  50. -50.], normal = [0. 1. 0.]
Plano 5: ponto = [ 50.  50. -50.], normal = [0. 0. 1.]


In [12]:
class Luz:
    def __init__(self, posicao, intensidade):
        self.posicao = posicao
        self.intensidade = intensidade
    
    def show(self):
        print("LUZ:")
        print(f'Posicao = {self.posicao}')
        print(f'Intensidade = {self.intensidade}')
    
    def get_pos(self):
        return self.posicao
    
    def get_int(self):
        return self.intensidade

In [13]:
luz = Luz(vetor(60, 120, 40), vetor(0.8, 0.8, 0.8))
luz.show()

LUZ:
Posicao = [ 60. 120.  40.]
Intensidade = [0.8 0.8 0.8]


In [14]:
class Cena:
    def __init__(self, camera, objetos, materiais, luzes):
        self.camera = camera
        self.objetos = objetos
        self.materiais = materiais
        self.luzes = luzes
    
    def show(self):
        print("CENA:")
        
        self.camera.show()
        
        for obj in self.objetos:
            obj.show()
        
        for mat in self.materiais:
            mat.show()
        
        for luz in self.luzes:
            luz.show()
    
    def shade(self, objeto, ponto, normal):
        kd = objeto.get_mat().get_kd()
        cor = vetor(0, 0, 0)
        
        for luz in self.luzes:
            vluz = unitario(luz.get_pos() - ponto)
            cosseno = dot(vluz, normal)
            
            if cosseno > TOL:
                cor += luz.get_int() * kd * cosseno
            
        return cor
    
    def ray_trace(self):
        w = self.camera.get_w()
        h = self.camera.get_h()
        origem = self.camera.get_eye()
        
        for y in range(h):           
            for x in range(w):
                direcao = self.camera.ray_to(x, y)
                ti = np.inf
                obji = None
                
                for obj in self.objetos:
                    t, objx = obj.intercepta(origem, direcao)
                    if t < ti:
                        ti = t
                        obji = objx
                
                if obji:
                    ponto = origem + ti * direcao
                    normal = obji.normal(ponto)
                    
                    rgb = self.shade(obji, ponto, normal)
                    #rgb = obji.get_mat().get_kd()
                    
                    cor = self.camera.get_pixel(x, y)
                    self.camera.set_pixel(x, y, cor + rgb)

In [15]:
objetos = [esfera, piso, parede]
materiais = [azul, amarelo]
luzes = [luz]
cena = Cena(camera, objetos, materiais, luzes)
cena.show()

CENA:
CAMERA:
fov = 90 d = 30
(w, h) = ( 300 , 300 )
(b, a) = ( 59.999999999999986 , 59.99999999999999 )
xe = [ 0.37139068  0.         -0.92847669]
ye = [ 0.32325409 -0.93743687  0.12930164]
ze = [-0.87038828 -0.34815531 -0.34815531]
ESFERA:
raio = 25
centro = [ 0. 20.  0.]
MATERIAL:
kd [0. 0. 1.]
POLIGONO:
Plano 0: ponto = [-80. -50. -50.], normal = [-1.  0.  0.]
Plano 1: ponto = [-80. -50. -50.], normal = [ 0. -1.  0.]
Plano 2: ponto = [-80. -50. -50.], normal = [ 0.  0. -1.]
Plano 3: ponto = [ 50. -45.  50.], normal = [1. 0. 0.]
Plano 4: ponto = [ 50. -45.  50.], normal = [0. 1. 0.]
Plano 5: ponto = [ 50. -45.  50.], normal = [0. 0. 1.]
POLIGONO:
Plano 0: ponto = [-80. -50. -60.], normal = [-1.  0.  0.]
Plano 1: ponto = [-80. -50. -60.], normal = [ 0. -1.  0.]
Plano 2: ponto = [-80. -50. -60.], normal = [ 0.  0. -1.]
Plano 3: ponto = [ 50.  50. -50.], normal = [1. 0. 0.]
Plano 4: ponto = [ 50.  50. -50.], normal = [0. 1. 0.]
Plano 5: ponto = [ 50.  50. -50.], normal = [0. 0. 1.]
MAT

In [16]:
cena.ray_trace()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
camera.imshow()